In [1]:
import time
import pandas as pd
import psycopg2
import datetime
from tqdm import trange
import matplotlib.pyplot as plt 
import matplotlib as mpl 
import seaborn as sns
import math
# %matplotlib inline 
import platform
import numpy as np
from matplotlib import font_manager, rc # plt.rcParams['axes.unicode_minus'] = False 
if platform.system() == 'Darwin': rc('font', family='AppleGothic') 
elif platform.system() == 'Windows': 
    path = "c:/Windows/Fonts/malgun.ttf" 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name) 
else: 
    print('Unknown system... sorry~~~~')
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
from tqdm import tqdm
from tqdm import trange
from datetime import datetime, timedelta


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))


In [3]:
# 시계열
from datetime import datetime as dt
from statsmodels.tsa.arima_model import ARIMA
from dateutil.relativedelta import relativedelta
from statsmodels.tsa.api import SimpleExpSmoothing, Holt, ExponentialSmoothing

In [4]:
    # row 생략 없이 출력
pd.set_option('display.max_rows', 1000)
# col 생략 없이 출력
pd.set_option('display.max_columns', 1000)
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
pd.options.display.float_format = '{:.5f}'.format

In [5]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

In [5]:
# 전체 매장 입출판재
total_shop_stock = pd.read_csv("total_shop_stock.csv")
total_shop_stock = total_shop_stock.iloc[ :,1:]

# 매장정보
shop_info_pd = pd.read_csv("shop_info_pd.csv")
shop_info_pd = shop_info_pd.iloc[ :,1:]

# bucket_dwalker 관련 매장입출판재(2019)
bucket_dwalker = pd.read_csv('bucket_dwalker.csv')
bucket_dwalker = bucket_dwalker.iloc[ :,1:]

# bucket_dwalker_v2 관련 매장입출판재
bucket_dwalker_v2 = pd.read_csv('bucket_dwalker_v2.csv')
bucket_dwalker_v2 = bucket_dwalker_v2.iloc[ :,1:]

# 아울렛 상설을 제외한 매장키 
shop_list_pd = pd.read_csv('shop_list.csv')
shop_list_pd = shop_list_pd.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 기준)
shop_weeks_buket_df = pd.read_csv('shop_weeks_buket.csv')
shop_weeks_buket_df = shop_weeks_buket_df.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터 (버킷 디워커 v2 기준)
shop_weeks_buket_v2_df = pd.read_csv('shop_weeks_buket_v2.csv')
shop_weeks_buket_v2_df = shop_weeks_buket_v2_df.iloc[ :,1:]

# 전체 매장 비어있는 주차 채운 데이터(2021 년 버킷디워커 v2 기준)
shop_weeks_buket_v2_2021_df = pd.read_csv('shop_weeks_buket_v2_2021_df.csv')
shop_weeks_buket_v2_2021_df = shop_weeks_buket_v2_2021_df.iloc[ :,1:]

# 전체 매장 비어있는것 채워넣은 입출판재 합쳐 놓은 것
shop_total_sales = pd.read_csv("shop_total_sales.csv")
shop_total_sales = shop_total_sales.iloc[ :,1:]

# 버킷디워커 물류 재고량
bk_stock_scsweek_pd = pd.read_csv("bk_stock_scsweek_pd.csv")
bk_stock_scsweek_pd = bk_stock_scsweek_pd.iloc[ :,1:]

# v2 버킷디워커 물류 재고량
v2_stock_scsweek_pd = pd.read_csv("v2_stock_scsweek_pd.csv")
v2_stock_scsweek_pd = v2_stock_scsweek_pd.iloc[ :,1:]


#v2 2021 물류 창고 재고량
v2_2021_stock_scsweek_pd = pd.read_csv('v2_2021_stock_scsweek_pd.csv')
v2_2021_stock_scsweek_pd = v2_2021_stock_scsweek_pd.iloc[ :,1:]


In [6]:
# 아이템별 재고 
# buket
be_shop_weeks_buket_df = pd.read_csv('be_shop_weeks_buket_df.csv')
be_shop_weeks_buket_df = be_shop_weeks_buket_df.iloc[ :,1:]

wh_shop_weeks_buket_df = pd.read_csv('wh_shop_weeks_buket_df.csv')
wh_shop_weeks_buket_df = wh_shop_weeks_buket_df.iloc[ :,1:]

bk_shop_weeks_buket_df = pd.read_csv('bk_shop_weeks_buket_df.csv')
bk_shop_weeks_buket_df = bk_shop_weeks_buket_df.iloc[ :,1:]

gr_shop_weeks_buket_df = pd.read_csv('gr_shop_weeks_buket_df.csv')
gr_shop_weeks_buket_df = gr_shop_weeks_buket_df.iloc[ :,1:]


# v2
ow_shop_weeks_buket_v2_df = pd.read_csv('ow_shop_weeks_buket_v2_df.csv')
ow_shop_weeks_buket_v2_df = ow_shop_weeks_buket_v2_df.iloc[ :,1:]

wh_shop_weeks_buket_v2_df = pd.read_csv('wh_shop_weeks_buket_v2_df.csv')
wh_shop_weeks_buket_v2_df = wh_shop_weeks_buket_v2_df.iloc[ :,1:]

be_shop_weeks_buket_v2_df = pd.read_csv('be_shop_weeks_buket_v2_df.csv')
be_shop_weeks_buket_v2_df = be_shop_weeks_buket_v2_df.iloc[ :,1:]


# v2 2021 년 기준 
be_shop_weeks_buket_2021_df = pd.read_csv('be_shop_weeks_buket_2021_df.csv')
be_shop_weeks_buket_2021_df = be_shop_weeks_buket_2021_df.iloc[ :,1:]


bk_shop_weeks_buket_2021_df = pd.read_csv('bk_shop_weeks_buket_2021_df.csv')
bk_shop_weeks_buket_2021_df = bk_shop_weeks_buket_2021_df.iloc[ :,1:]

cr_shop_weeks_buket_2021_df = pd.read_csv('cr_shop_weeks_buket_2021_df.csv')
cr_shop_weeks_buket_2021_df = cr_shop_weeks_buket_2021_df.iloc[ :,1:]

iv_shop_weeks_buket_2021_df = pd.read_csv('iv_shop_weeks_buket_2021_df.csv')
iv_shop_weeks_buket_2021_df = iv_shop_weeks_buket_2021_df.iloc[ :,1:]

lg_shop_weeks_buket_2021_df = pd.read_csv('lg_shop_weeks_buket_2021_df.csv')
lg_shop_weeks_buket_2021_df = lg_shop_weeks_buket_2021_df.iloc[ :,1:]

ow_shop_weeks_buket_2021_df = pd.read_csv('ow_shop_weeks_buket_2021_df.csv')
ow_shop_weeks_buket_2021_df = ow_shop_weeks_buket_2021_df.iloc[ :,1:]

wh_shop_weeks_buket_2021_df = pd.read_csv('wh_shop_weeks_buket_2021_df.csv')
wh_shop_weeks_buket_2021_df = wh_shop_weeks_buket_2021_df.iloc[ :,1:]

ww_shop_weeks_buket_2021_df = pd.read_csv('ww_shop_weeks_buket_2021_df.csv')
ww_shop_weeks_buket_2021_df = ww_shop_weeks_buket_2021_df.iloc[ :,1:]



In [7]:
# DB 연결 부분
uswer = 'dt.intern'
host = '10.80.0.234'
port = '5439'
dbname = 'fnf'
con = psycopg2.connect(dbname='fnf',host='10.80.0.234', port='5439', user = 'dt.intern', password='DTintn0926')
cur = con.cursor()

In [8]:
def mae(prediction, correct):
    prediction = np.array(prediction)
    correct = np.array(correct)
    
    difference = correct - prediction
    abs_val = abs(difference)
    
    score = abs_val.mean()
    
    return score

# mae_scorer = make_scorer(mae)
# mae_scorer

In [9]:
def make_ewm_arr(data_frame, span_num):
    arr_ewm = np.array([])
#     for i in data_frame.store_id.unique():
    df_set = data_frame
        # 여기에서 지정하는 span값은 위 수식에서 N에 해당한다.
    arr_ewm = np.concatenate((arr_ewm, df_set.week_sale_qty.ewm(span=span_num).mean().values) )
    
    return arr_ewm

In [180]:
# 2019년 버킷디워커 물류 수량
start = time.time()

bk_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270',
'X19SDXSH09911GR230','X19SDXSH09911GR235','X19SDXSH09911GR240','X19SDXSH09911GR260','X19SDXSH09911GR265','X19SDXSH09911GR270',
'X19SDXSH09911WH230','X19SDXSH09911WH235','X19SDXSH09911WH240','X19SDXSH09911WH260','X19SDXSH09911WH265','X19SDXSH09911WH270',
'X19SDXSH09911BK230','X19SDXSH09911BK235','X19SDXSH09911BK240','X19SDXSH09911BK260','X19SDXSH09911BK265','X19SDXSH09911BK270') 
AND weekkey BETWEEN To_date('2018-12-31', 'YYYY-MM-DD') AND To_date('2019-12-31', 'YYYY-MM-DD')


'''


cur.execute(bk_stock_scsweek)
bk_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

bk_stock_scsweek_pd = pd.DataFrame(bk_stock_scsweek)
bk_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 0.10473084449768066


In [184]:
bk_stock_scsweek_pd.to_csv('bk_stock_scsweek_pd.csv')

In [146]:
# 2020년 v2 버킷디워커 물류 수량
start = time.time()

v2_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN ('X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270',
'X20SDXSHA1011WH230','X20SDXSHA1011WH235','X20SDXSHA1011WH240','X20SDXSHA1011WH260','X20SDXSHA1011WH265','X20SDXSHA1011WH270',
'X20SDXSHA3011BE230','X20SDXSHA3011BE235','X20SDXSHA3011BE240','X20SDXSHA3011BE260','X20SDXSHA3011BE265','X20SDXSHA3011BE270'
) AND weekkey BETWEEN To_date('2019-11-18', 'YYYY-MM-DD') AND To_date('2020-12-31', 'YYYY-MM-DD')


'''


cur.execute(v2_stock_scsweek)
v2_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

v2_stock_scsweek_pd = pd.DataFrame(v2_stock_scsweek)
v2_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 0.1366875171661377


In [185]:
v2_stock_scsweek_pd.to_csv('v2_stock_scsweek_pd.csv')

In [58]:
# 2021년 v2 버킷디워커 물류 수량
start = time.time()

v2_2021_stock_scsweek = '''

SELECT weekkey ,scskey, week_accept_qty ,week_wsl_takeout_qty, wms_stock_qty ,stock_qty,domestic_sale_qty
FROM ax.vw_f_scsweek vfs 
WHERE scskey  IN (
'X21SDXSHA1111BE230','X21SDXSHA1111BE235','X21SDXSHA1111BE240','X21SDXSHA1111BE260','X21SDXSHA1111BE260','X21SDXSHA1111BE270',
'X21SDXSHA1111BK230','X21SDXSHA1111BK235','X21SDXSHA1111BK240','X21SDXSHA1111BK260','X21SDXSHA1111BK260','X21SDXSHA1111BK270',
'X21SDXSHA1111CR230','X21SDXSHA1111CR235','X21SDXSHA1111CR240','X21SDXSHA1111CR260','X21SDXSHA1111CR260','X21SDXSHA1111CR270',
'X21SDXSHA1111IV230','X21SDXSHA1111IV235','X21SDXSHA1111IV240','X21SDXSHA1111IV260','X21SDXSHA1111IV260','X21SDXSHA1111IV270',
'X21SDXSHA1111LG230','X21SDXSHA1111LG235','X21SDXSHA1111LG240','X21SDXSHA1111LG260','X21SDXSHA1111LG260','X21SDXSHA1111LG270',
'X21SDXSHA1111OW230','X21SDXSHA1111OW235','X21SDXSHA1111OW240','X21SDXSHA1111OW260','X21SDXSHA1111OW260','X21SDXSHA1111OW270',
'X21SDXSHA1111WH230','X21SDXSHA1111WH235','X21SDXSHA1111WH240','X21SDXSHA1111WH260','X21SDXSHA1111WH260','X21SDXSHA1111WH270',
'X21SDXSHA1111WW230','X21SDXSHA1111WW235','X21SDXSHA1111WW240','X21SDXSHA1111WW260','X21SDXSHA1111WW260','X21SDXSHA1111WW270'
) AND weekkey BETWEEN To_date('2020-11-18', 'YYYY-MM-DD') AND To_date('2021-04-23', 'YYYY-MM-DD')


'''


cur.execute(v2_2021_stock_scsweek)
v2_2021_stock_scsweek = cur.fetchall()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

v2_2021_stock_scsweek_pd = pd.DataFrame(v2_2021_stock_scsweek)
v2_2021_stock_scsweek_pd.columns = ['weekkey' ,'scskey', 'week_accept_qty' ,'week_wsl_takeout_qty', 'wms_stock_qty' ,'stock_qty','domestic_sale_qty']

time : 5.051944732666016


In [62]:
v2_2021_stock_scsweek_pd.to_csv('v2_2021_stock_scsweek_pd.csv')

In [61]:
v2_2021_stock_scsweek_pd.sort_values('weekkey')

,weekkey,scskey,week_accept_qty,week_wsl_takeout_qty,wms_stock_qty,stock_qty,domestic_sale_qty
0,2020-11-23,X21SDXSHA1111BE235,0,0,0,0,0
37,2020-11-23,X21SDXSHA1111BK240,0,0,0,0,0
53,2020-11-23,X21SDXSHA1111OW235,0,0,0,0,0
57,2020-11-23,X21SDXSHA1111OW240,0,0,0,0,0
353,2020-11-23,X21SDXSHA1111IV235,0,0,0,0,0
68,2020-11-23,X21SDXSHA1111BE260,0,0,0,0,0
357,2020-11-23,X21SDXSHA1111IV270,0,0,0,0,0
72,2020-11-23,X21SDXSHA1111BK235,0,0,0,0,0
76,2020-11-23,X21SDXSHA1111BK260,0,0,0,0,0
361,2020-11-23,X21SDXSHA1111OW230,0,0,0,0,0


# 재고가 0 인 지점

## 버킷 디워커 재고량 0 인 지점

In [10]:
shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='대리점']
bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

bk_shop_qty = pd.DataFrame()

for i in bk_shopkey_unique:
    uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
    for j in bk_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]


<ipython-input-10-83b7ffe6a69c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-10-83b7ffe6a69c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [11]:
## 버킷디워커 재고량 0 인 지점과 실 재고량 확인 

In [12]:
bk_zero_point = bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0]
bk_zero_point = bk_zero_point.sort_values(['weekkey','scskey'])

In [13]:
bk_stock_scsweek_pd['weekkey'] = pd.to_datetime(bk_stock_scsweek_pd['weekkey'])
bk_zero_point['weekkey'] = pd.to_datetime(bk_zero_point['weekkey'])

In [14]:
def shop_weeks_buket_function(shop_weeks_buket_df):
    shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='대리점']
    bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
    bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
    shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

    bk_shop_qty = pd.DataFrame()

    for i in bk_shopkey_unique:
        uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
        for j in bk_scskey_unique:
            uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
            ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
            uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
            ewm_value = np.append([0],ewm_value,axis=0)
            uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
            for i in range(0,len(ewm_value)):
    #             ewm_value[i] = round(ewm_value[i] , 0)+1
                ewm_value[i] = math.ceil(ewm_value[i])+1

            uniqe_shop_scskey['qty'] = ewm_value[:-1]
            bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

    bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]
    return bk_zero


## 버킷 디워커 창고 재고량 있는지 확인

In [15]:

bk_zero_point = bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0]
bk_zero_point = bk_zero_point.sort_values(['weekkey','scskey'])
bk_stock_scsweek_pd['weekkey'] = pd.to_datetime(bk_stock_scsweek_pd['weekkey'])
bk_zero_point['weekkey'] = pd.to_datetime(bk_zero_point['weekkey'])

# 실 재고량과 비교 
bk_true_zero_count = 0
for i in bk_zero_point.weekkey.unique(): # 날짜 
    for j in bk_zero_point.scskey.unique(): # scskey
        bk_zero_point_check = bk_zero_point[(bk_zero_point['weekkey']==i) & (bk_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
        if len(bk_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
            break
            
        # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
        wms_stock = bk_stock_scsweek_pd[(bk_stock_scsweek_pd['weekkey']==bk_zero_point_check.iloc[0].weekkey) & (bk_stock_scsweek_pd.scskey ==bk_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

        for z in range(0,len(bk_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
            if wms_stock - bk_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만 
                bk_true_zero_count+=1                          # count +1 하고
                wms_stock-= bk_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
            else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                break

In [16]:
# 재고 반영
bk_true_zero_count

900

In [17]:
# 재고 없이 로직으로
len(bk_zero_point)

1373

In [18]:
# 재고가 0 인 지점 
len(bk_zero)

1654

In [19]:
# 실 재고로 비교하는 함수
def bk_stock_zero_function(bk_zero):
    bk_zero_point = bk_zero[bk_zero.qty - bk_zero.week_sale_qty>0]
    bk_zero_point = bk_zero_point.sort_values(['weekkey','scskey'])
    bk_stock_scsweek_pd['weekkey'] = pd.to_datetime(bk_stock_scsweek_pd['weekkey'])
    bk_zero_point['weekkey'] = pd.to_datetime(bk_zero_point['weekkey'])

    # 실 재고량과 비교 
    bk_true_zero_count = 0
    for i in bk_zero_point.weekkey.unique(): # 날짜 
        for j in bk_zero_point.scskey.unique(): # scskey
            bk_zero_point_check = bk_zero_point[(bk_zero_point['weekkey']==i) & (bk_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
            if len(bk_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
                break

            # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
            wms_stock = bk_stock_scsweek_pd[(bk_stock_scsweek_pd['weekkey']==bk_zero_point_check.iloc[0].weekkey) & (bk_stock_scsweek_pd.scskey ==bk_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

            for z in range(0,len(bk_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
                if wms_stock - bk_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만 
                    bk_true_zero_count+=1                          # count +1 하고
                    wms_stock-= bk_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
                else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                    break
    zero_count = len(bk_zero) - bk_true_zero_count
    return zero_count

## v2 재고량 0인 지점 확인

In [30]:
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='대리점']
shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey'] != 'X10047']
v2_shopkey_unique =  shop_weeks_buket_v2_df_dep.shopkey.unique()
v2_scskey_unique =  shop_weeks_buket_v2_df_dep.scskey.unique()

v2_shop_qty = pd.DataFrame()

for i in v2_shopkey_unique:
    uniqe_shop = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
    for j in v2_scskey_unique:
        uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
        ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
        uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
        ewm_value = np.append([0],ewm_value,axis=0)
        uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
        for i in range(0,len(ewm_value)):
#             ewm_value[i] = round(ewm_value[i] , 0)+1
            ewm_value[i] = math.ceil(ewm_value[i])+1

        uniqe_shop_scskey['qty'] = ewm_value[:-1]
        v2_shop_qty = pd.concat([v2_shop_qty, uniqe_shop_scskey], axis=0)
v2_zero = v2_shop_qty[(v2_shop_qty['stock_qty']==0) & (v2_shop_qty['takeout_qty']>0)]

<ipython-input-30-2ad021129a44>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-30-2ad021129a44>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]


In [31]:
def shop_weeks_buket_function(shop_weeks_buket_df):
    shop_weeks_buket_df_dep = shop_weeks_buket_df[shop_weeks_buket_df['type_zone_nm']=='대리점']
    bk_shopkey_unique =  shop_weeks_buket_df_dep.shopkey.unique()
    bk_scskey_unique =  shop_weeks_buket_df_dep.scskey.unique()
    shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']

    bk_shop_qty = pd.DataFrame()

    for i in bk_shopkey_unique:
        uniqe_shop = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey']==i]
        for j in bk_scskey_unique:
            uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
            ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
            uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
            ewm_value = np.append([0],ewm_value,axis=0)
            uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
            for i in range(0,len(ewm_value)):
    #             ewm_value[i] = round(ewm_value[i] , 0)+1
                ewm_value[i] = math.ceil(ewm_value[i])+1

            uniqe_shop_scskey['qty'] = ewm_value[:-1]
            bk_shop_qty = pd.concat([bk_shop_qty, uniqe_shop_scskey], axis=0)

    bk_zero = bk_shop_qty[(bk_shop_qty['stock_qty']==0) & (bk_shop_qty['takeout_qty']>0)]
    return bk_zero , bk_shop_qty


In [32]:
def shop_weeks_buket_v2_function(shop_weeks_buket_v2_df):
    shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df[shop_weeks_buket_v2_df['type_zone_nm']=='대리점']
    shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey'] != 'X10047']
    v2_shopkey_unique =  shop_weeks_buket_v2_df_dep.shopkey.unique()
    v2_scskey_unique =  shop_weeks_buket_v2_df_dep.scskey.unique()

    v2_shop_qty = pd.DataFrame()

    for i in v2_shopkey_unique:
        uniqe_shop = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
        for j in v2_scskey_unique:
            uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
            ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
            uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
            ewm_value = np.append([0],ewm_value,axis=0)
            uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
            for i in range(0,len(ewm_value)):
    #             ewm_value[i] = round(ewm_value[i] , 0)+1
                ewm_value[i] = math.ceil(ewm_value[i])+1

            uniqe_shop_scskey['qty'] = ewm_value[:-1]
            v2_shop_qty = pd.concat([v2_shop_qty, uniqe_shop_scskey], axis=0)
    v2_zero = v2_shop_qty[(v2_shop_qty['stock_qty']==0) & (v2_shop_qty['takeout_qty']>0)]
    return v2_zero,v2_shop_qty


## v2 버킷 디워커 창고 재고량 있는지 확인

In [33]:
## 버킷 디워커 v2  창고 재고량 있는지 확인

v2_zero_point = v2_zero[v2_zero.qty - v2_zero.week_sale_qty>0]
v2_zero_point = v2_zero_point.sort_values(['weekkey','scskey'])

In [34]:
v2_stock_scsweek_pd['weekkey'] = pd.to_datetime(v2_stock_scsweek_pd['weekkey'])
v2_zero_point['weekkey'] = pd.to_datetime(v2_zero_point['weekkey'])

In [35]:
# 실 재고량과 비교 
v2_true_zero_count = 0
for i in v2_zero_point.weekkey.unique(): # 날짜 
    for j in v2_zero_point.scskey.unique(): # scskey
        v2_zero_point_check = v2_zero_point[(v2_zero_point['weekkey']==i) & (v2_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
        if len(v2_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
            break
            
        # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
        v2_wms_stock = v2_stock_scsweek_pd[(v2_stock_scsweek_pd['weekkey']==v2_zero_point_check.iloc[0].weekkey) & (v2_stock_scsweek_pd.scskey ==v2_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

        for z in range(0,len(v2_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
            if v2_wms_stock - v2_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만 
                v2_true_zero_count+=1                          # count +1 하고
                v2_wms_stock-= v2_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
            else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                break

In [36]:
v2_true_zero_count

10

In [37]:
len(v2_zero_point)

44

In [38]:
len(v2_zero)

190

In [39]:
# 실 재고로 비교하는 함수
def v2_stock_zero_function(v2_zero):
    v2_zero_point = v2_zero[v2_zero.qty - v2_zero.week_sale_qty>0]
    v2_zero_point = v2_zero_point.sort_values(['weekkey','scskey'])
    v2_stock_scsweek_pd['weekkey'] = pd.to_datetime(v2_stock_scsweek_pd['weekkey'])
    v2_zero_point['weekkey'] = pd.to_datetime(v2_zero_point['weekkey'])

        # 실 재고량과 비교 
    v2_true_zero_count = 0
    for i in v2_zero_point.weekkey.unique(): # 날짜 
        for j in v2_zero_point.scskey.unique(): # scskey
            v2_zero_point_check = v2_zero_point[(v2_zero_point['weekkey']==i) & (v2_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
            if len(v2_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
                break

            # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
            v2_wms_stock = v2_stock_scsweek_pd[(v2_stock_scsweek_pd['weekkey']==v2_zero_point_check.iloc[0].weekkey) & (v2_stock_scsweek_pd.scskey ==v2_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

            for z in range(0,len(v2_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
                if v2_wms_stock - v2_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만 
                    v2_true_zero_count+=1                          # count +1 하고
                    v2_wms_stock-= v2_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
                else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                    break
    zero_count = len(v2_zero) - v2_true_zero_count
    return zero_count

# 2021_v2로 비교

In [40]:
# 재고량이 0 인곳 
def shop_weeks_buket_v2_2021_function(shop_weeks_buket_v2_2021_df):
    shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_2021_df[shop_weeks_buket_v2_2021_df['type_zone_nm']=='대리점']
    shop_weeks_buket_v2_df_dep = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey'] != 'X10047']
    v2_shopkey_unique =  shop_weeks_buket_v2_df_dep.shopkey.unique()
    v2_scskey_unique =  shop_weeks_buket_v2_df_dep.scskey.unique()

    v2_shop_qty = pd.DataFrame()

    for i in v2_shopkey_unique:
        uniqe_shop = shop_weeks_buket_v2_df_dep[shop_weeks_buket_v2_df_dep['shopkey']==i]
        for j in v2_scskey_unique:
            uniqe_shop_scskey = uniqe_shop[uniqe_shop['scskey']==j]
            ewm_value = make_ewm_arr(uniqe_shop_scskey, 2)
            uniqe_shop_scskey.insert(11, 'amount_2ewm', ewm_value)
            ewm_value = np.append([0],ewm_value,axis=0)
            uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
            for i in range(0,len(ewm_value)):
    #             ewm_value[i] = round(ewm_value[i] , 0)+1
                ewm_value[i] = math.ceil(ewm_value[i])+1

            uniqe_shop_scskey['qty'] = ewm_value[:-1]
            v2_shop_qty = pd.concat([v2_shop_qty, uniqe_shop_scskey], axis=0)
    v2_zero = v2_shop_qty[(v2_shop_qty['stock_qty']==0) & (v2_shop_qty['takeout_qty']>0)]
    return v2_zero


In [129]:
# 실 재고로 비교하는 함수
def v2_2021_stock_zero_function(v2_zero,v2_2021_stock_scsweek_pd):
    v2_zero_point = v2_zero[v2_zero.qty - v2_zero.week_sale_qty>0]
    v2_zero_point = v2_zero_point.sort_values(['weekkey','scskey'])
    v2_2021_stock_scsweek_pd['weekkey'] = pd.to_datetime(v2_2021_stock_scsweek_pd['weekkey'])
    v2_zero_point['weekkey'] = pd.to_datetime(v2_zero_point['weekkey'])
    
    true_list = []
        # 실 재고량과 비교 
    v2_true_zero_count = 0
    for i in v2_zero_point.weekkey.unique(): # 날짜 
        for j in v2_zero_point.scskey.unique(): # scskey
            v2_zero_point_check = v2_zero_point[(v2_zero_point['weekkey']==i) & (v2_zero_point['scskey']==j)] # 해당 날짜의 scskey가 있는지 확인 
            if len(v2_zero_point_check) == 0 : # 없으면 다음으로 넘기고 
                break

            # 있을 경우에 해당 날짜 scskey 의 창고 물량을 확인
            v2_wms_stock = v2_2021_stock_scsweek_pd[(v2_2021_stock_scsweek_pd['weekkey']==v2_zero_point_check.iloc[0].weekkey) & (v2_2021_stock_scsweek_pd.scskey ==v2_zero_point_check.iloc[0].scskey) ][['wms_stock_qty']].iloc[0][0] 

            for z in range(0,len(v2_zero_point_check)):            # 창고내 해당 물건의 재고량 - 재고 0 이였던 지점의 qty 
                if v2_wms_stock - v2_zero_point_check.iloc[z].qty>0:  # 1보다  클 경우만
                    v2_true_zero_count+=1                          # count +1 하고
                    v2_wms_stock-= v2_zero_point_check.iloc[z].qty    # wms의 값 또한 빼기한다.
                else:                                              # 1보다 작으면 break 하고 다음 for 문으로 넘긴다.
                    true_list.append(v2_zero_point_check.iloc[z])
    zero_count = len(v2_zero) - v2_true_zero_count
#     return zero_count , true_list
    return zero_count 

# 과잉 재고가 되지 않는지 (MAE 값 비교)

In [42]:
## 버킷 디워커 MAE 값 비교

In [43]:
def over_charge(bk_shop_qty):
    bk_shopkey_unique = bk_shop_qty.shopkey.unique()
    bk_scskey_unique = bk_shop_qty.scskey.unique()
    bk_ori_mae_list=[]
    bk_imp_mae_list=[]

    bk_score_list = []
    bk_score_list2 = []

    for i in bk_shopkey_unique:
        for j in bk_scskey_unique:
    #         print(test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:])
            bk_ori_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[:-1] ))
            bk_imp_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:-1] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[1:-1] ))

            bk_score_list.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[:-1] ).dropna()))
            bk_score_list2.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[1:] ).dropna()))
    return np.mean(bk_ori_mae_list),np.mean(bk_imp_mae_list)
    

In [44]:
def over_charge(bk_shop_qty):
    bk_shopkey_unique = bk_shop_qty.shopkey.unique()
    bk_scskey_unique = bk_shop_qty.scskey.unique()
    bk_ori_mae_list=[]
    bk_imp_mae_list=[]

    bk_score_list = []
    bk_score_list2 = []
    for i in bk_shopkey_unique:
        count = 0
        for j in bk_scskey_unique:
            if bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].takeout_qty[count:].iloc[0]==0:
                count +=1
                continue
            bk_ori_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[count+1:] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[count:-1] ))
            bk_imp_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[count+1:-1] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[count+1:-1] ))

            bk_score_list.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[count+1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[count:-1] ).dropna()))
            bk_score_list2.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[count+1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[count+1:] ).dropna()))
    return np.mean(bk_ori_mae_list),np.mean(bk_imp_mae_list)
    

In [45]:
bk_shopkey_unique = bk_shop_qty.shopkey.unique()
bk_scskey_unique = bk_shop_qty.scskey.unique()
bk_ori_mae_list=[]
bk_imp_mae_list=[]

bk_score_list = []
bk_score_list2 = []

for i in bk_shopkey_unique:
    for j in bk_scskey_unique:
#         print(test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:])
        bk_ori_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[:-1] ))
        bk_imp_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:-1] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[1:-1] ))

        bk_score_list.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[:-1] ).dropna()))
        bk_score_list2.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[1:] ).dropna()))

#         break
#     break
        
    

In [46]:
np.mean(bk_ori_mae_list)

4.123214285714286

In [47]:
np.mean(bk_imp_mae_list)

1.7797852474323066

In [48]:
## v2 버킷 디워커 MAE 값 비교

In [49]:
v2_shopkey_unique = v2_shop_qty.shopkey.unique()
v2_scskey_unique = v2_shop_qty.scskey.unique()
v2_ori_mae_list=[]
v2_imp_mae_list=[]

v2_score_list = []
v2_score_list2 = []

for i in v2_shopkey_unique:
    for j in v2_scskey_unique:
#         print(test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:])
        v2_ori_mae_list.append(mae( v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:] ,v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].stock_qty[:-1] ))
        v2_imp_mae_list.append(mae( v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:-1] ,v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].qty[1:-1] ))

        v2_score_list.append ( sum((v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:]-v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].stock_qty[:-1] ).dropna()))
        v2_score_list2.append ( sum((v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:]-v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].qty[1:] ).dropna()))

#         break
#     break
        
    

In [50]:
np.mean(v2_ori_mae_list)

3.450656814449918

In [51]:
np.mean(v2_imp_mae_list)

1.6941520467836255

In [52]:
## 재고 총 개수 

In [53]:
bk_shop_qty.stock_qty.sum()

90369

In [54]:
bk_shop_qty.qty.sum()

42415.0

In [55]:
v2_shop_qty.stock_qty.sum()

91095

In [56]:
v2_shop_qty.qty.sum()  # ow_shop_weeks_v2_df_zero

49054.0

In [57]:
v2_shop_qty

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
59,2019-11-18,1,1,0,0,1,X50088,0.00000,X20SDXSHA5011OW230,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
60,2019-11-25,0,1,0,0,1,X50088,0.00000,X20SDXSHA5011OW230,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
61,2019-12-02,0,1,0,0,1,X50088,0.00000,X20SDXSHA5011OW230,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
62,2019-12-09,0,1,0,0,1,X50088,0.00000,X20SDXSHA5011OW230,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
63,2019-12-16,0,1,0,0,1,X50088,0.00000,X20SDXSHA5011OW230,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50617,2020-11-30,0,29,0,24,5,X50002,1.00000,X20SDXSHA5011OW270,대전은행(대-위),디스커버리 대전은행점,0.00000,XH0002,일반매장,대리점,대리점,대전,0.00001,2.00000
50618,2020-12-07,0,29,0,24,5,X50002,1.00000,X20SDXSHA5011OW270,대전은행(대-위),디스커버리 대전은행점,0.00000,XH0002,일반매장,대리점,대리점,대전,0.00000,2.00000
50619,2020-12-14,0,29,0,24,5,X50002,1.00000,X20SDXSHA5011OW270,대전은행(대-위),디스커버리 대전은행점,0.00000,XH0002,일반매장,대리점,대리점,대전,0.00000,2.00000
50620,2020-12-21,0,29,0,24,5,X50002,1.00000,X20SDXSHA5011OW270,대전은행(대-위),디스커버리 대전은행점,0.00000,XH0002,일반매장,대리점,대리점,대전,0.00000,2.00000


In [58]:
ow_shop_weeks_buket_v2_df[(ow_shop_weeks_buket_v2_df['type_zone_nm']=='백화점')&(ow_shop_weeks_buket_v2_df['shopkey'] != 'X10047')]
# shop_weeks_buket_df_dep = shop_weeks_buket_df_dep[shop_weeks_buket_df_dep['shopkey'] != 'X10047']


,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2019-11-18,1,1,0,0,1,X10052,0.00000,X20SDXSHA5011OW230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
1,2019-11-25,0,1,0,0,1,X10052,0.00000,X20SDXSHA5011OW230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
2,2019-12-02,0,1,0,0,1,X10052,0.00000,X20SDXSHA5011OW230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
3,2019-12-09,0,1,0,0,1,X10052,0.00000,X20SDXSHA5011OW230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
4,2019-12-16,0,1,0,0,1,X10052,0.00000,X20SDXSHA5011OW230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50440,2020-11-30,0,24,0,21,3,X10020,1.00000,X20SDXSHA5011OW270,롯데청량리,디스커버리 롯데 청량리점,XH0209,일반매장,백화점특정,백화점,서울
50441,2020-12-07,0,24,0,21,3,X10020,1.00000,X20SDXSHA5011OW270,롯데청량리,디스커버리 롯데 청량리점,XH0209,일반매장,백화점특정,백화점,서울
50442,2020-12-14,0,24,0,21,3,X10020,1.00000,X20SDXSHA5011OW270,롯데청량리,디스커버리 롯데 청량리점,XH0209,일반매장,백화점특정,백화점,서울
50443,2020-12-21,0,24,0,21,3,X10020,1.00000,X20SDXSHA5011OW270,롯데청량리,디스커버리 롯데 청량리점,XH0209,일반매장,백화점특정,백화점,서울


In [59]:
v2_shopkey_unique = v2_shop_qty.shopkey.unique()
v2_scskey_unique = v2_shop_qty.scskey.unique()
v2_ori_mae_list=[]
v2_imp_mae_list=[]

v2_score_list = []
v2_score_list2 = []

for i in v2_shopkey_unique:
    for j in v2_scskey_unique:
#         print(test1[(test1['shopkey']==i) & (test1['scskey']==j)].week_takeout_qty[1:])
        v2_ori_mae_list.append(mae( v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:] ,v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].stock_qty[:-1] ))
        v2_imp_mae_list.append(mae( v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:-1] ,v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].qty[1:-1] ))

        v2_score_list.append ( sum((v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:]-v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].stock_qty[:-1] ).dropna()))
        v2_score_list2.append ( sum((v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].week_takeout_qty[1:]-v2_shop_qty[(v2_shop_qty['shopkey']==i) & (v2_shop_qty['scskey']==j)].qty[1:] ).dropna()))

#         break
#     break
        
    

# 매장별 부족한 입출판재 채우는 로직 

In [51]:
# df_INNER_JOIN['weekkey'] = pd.to_datetime(df_INNER_JOIN['weekkey'])

NameError: name 'df_INNER_JOIN' is not defined

In [ ]:
# bucket_dwalker = df_INNER_JOIN[(df_INNER_JOIN['partkey'].str.contains("X19SDXSH09911")) & (df_INNER_JOIN['weekkey'] < '2019-12-31' ) ]

In [ ]:
# bucket_dwalker_v2 = df_INNER_JOIN[df_INNER_JOIN['partkey'].str.contains("X20SDXSHA5011|X20SDXSHA3011|X20SDXSHA1011|X20SDXSHA1011") & (df_INNER_JOIN['weekkey'] < '2020-12-31' )]

In [60]:
# 빠져있는 주차 채워주는 로직

def zero_fill(shopkey, base, scskey_list,item):
    total=[]
    for col in base.columns:
        base[col].values[:] = 0
    for scskey in scskey_list:
        for shop in shopkey:
            frame = base.reset_index()
            target = item[(item['scskey']==scskey) & (item['shopkey']==shop )].groupby('weekkey').sum().reset_index()
            target = target.reset_index()

            for i in range(0,len(target)):
                for j in range(0,len(frame)):
                        if target.iloc[i].weekkey == frame.iloc[j].weekkey:
                            frame.iloc[j] = target.iloc[i]
                            break
            for i in range(1,len(frame)):
                if frame.iloc[i].takeout_qty ==0 and frame.iloc[i].sale_qty==0:
                    frame.iloc[i] = [ frame.iloc[i]['weekkey'], 0, frame.iloc[i-1]['takeout_qty'], 0, frame.iloc[i-1]['sale_qty'], frame.iloc[i-1]['stock_qty']]
            frame['shopkey'] =shop
            frame['ratio'] = 0
            for i in range(0,len(frame)):
                total_sale_qty= frame['week_sale_qty'].sum()
                a = frame.iloc[i]
                if a['sale_qty']==0 :
                    frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 0]
                else:
                    if a['sale_qty'] / total_sale_qty > 1:
                        frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , 1]
                    else:
                        frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]
            else:
                frame['scskey']=scskey
            frame = pd.merge(frame, shop_info_pd, left_on='shopkey', right_on='shopkey', how='inner')
            total.append(frame)
    return total


In [61]:
## 버킷 디워커

In [62]:
# BE
scskey_list = ['X19SDXSH09911BE230','X19SDXSH09911BE235','X19SDXSH09911BE240','X19SDXSH09911BE260','X19SDXSH09911BE265','X19SDXSH09911BE270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911BE240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)
be_shop_weeks_buket_df = pd.concat(shop_weeks_buket)

In [63]:
# GR
scskey_list = ['X19SDXSH09911GR230','X19SDXSH09911GR235','X19SDXSH09911GR240','X19SDXSH09911GR260','X19SDXSH09911GR265','X19SDXSH09911GR270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911GR240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)
gr_shop_weeks_buket_df = pd.concat(shop_weeks_buket)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-60-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [64]:
# WH
scskey_list = ['X19SDXSH09911WH230','X19SDXSH09911WH235','X19SDXSH09911WH240','X19SDXSH09911WH260','X19SDXSH09911WH265','X19SDXSH09911WH270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911WH240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)
wh_shop_weeks_buket_df = pd.concat(shop_weeks_buket)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-60-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [65]:
# BK

scskey_list = ['X19SDXSH09911BK230','X19SDXSH09911BK235','X19SDXSH09911BK240','X19SDXSH09911BK260','X19SDXSH09911BK265','X19SDXSH09911BK270']
frame = bucket_dwalker[(bucket_dwalker['scskey']=='X19SDXSH09911BK240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_shop_key = bucket_dwalker['shopkey'].unique()


shop_weeks_buket = zero_fill(bucket_dwalker_shop_key, frame,scskey_list,bucket_dwalker)
bk_shop_weeks_buket_df = pd.concat(shop_weeks_buket)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


## v2 버킷 디워커

In [66]:
# OW 
scskey_list = ['X20SDXSHA5011OW230','X20SDXSHA5011OW235','X20SDXSHA5011OW240','X20SDXSHA5011OW260','X20SDXSHA5011OW265','X20SDXSHA5011OW270']
frame = bucket_dwalker_v2[(bucket_dwalker_v2['scskey']=='X20SDXSHA5011OW240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_v2_shop_key = bucket_dwalker_v2['shopkey'].unique()


shop_weeks_buket_v2 = zero_fill(bucket_dwalker_v2_shop_key, frame,scskey_list,bucket_dwalker_v2)
ow_shop_weeks_buket_v2_df = pd.concat(shop_weeks_buket_v2)

In [67]:
# wh
scskey_list = ['X20SDXSHA1011WH230','X20SDXSHA1011WH235','X20SDXSHA1011WH240','X20SDXSHA1011WH260','X20SDXSHA1011WH265','X20SDXSHA1011WH270']
frame = bucket_dwalker_v2[(bucket_dwalker_v2['scskey']=='X20SDXSHA1011WH240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_v2_shop_key = bucket_dwalker_v2['shopkey'].unique()


shop_weeks_buket_v2 = zero_fill(bucket_dwalker_v2_shop_key, frame,scskey_list,bucket_dwalker_v2)
wh_shop_weeks_buket_v2_df = pd.concat(shop_weeks_buket_v2)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-60-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [68]:
# BE
scskey_list = ['X20SDXSHA3011BE230','X20SDXSHA3011BE235','X20SDXSHA3011BE240','X20SDXSHA3011BE260','X20SDXSHA3011BE265','X20SDXSHA3011BE270']
frame = bucket_dwalker_v2[(bucket_dwalker_v2['scskey']=='X20SDXSHA3011BE240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
bucket_dwalker_v2_shop_key = bucket_dwalker_v2['shopkey'].unique()


shop_weeks_buket_v2 = zero_fill(bucket_dwalker_v2_shop_key, frame,scskey_list,bucket_dwalker_v2)
be_shop_weeks_buket_v2_df = pd.concat(shop_weeks_buket_v2)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-60-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [69]:
be_shop_weeks_buket_v2_df

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2019-11-18,1,1,0,0,1,X10056,0.00000,X20SDXSHA3011BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
1,2019-11-25,0,1,1,1,0,X10056,0.10000,X20SDXSHA3011BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
2,2019-12-02,4,5,1,2,3,X10056,0.20000,X20SDXSHA3011BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
3,2019-12-09,1,6,1,3,3,X10056,0.30000,X20SDXSHA3011BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
4,2019-12-16,0,6,0,3,3,X10056,0.30000,X20SDXSHA3011BE230,갤러리아진주,디스커버리 갤러리아 진주점,XH0247,일반매장,백화점특정,백화점,경남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,2020-11-30,0,16,0,13,3,X50002,0.92857,X20SDXSHA3011BE270,대전은행(대-위),디스커버리 대전은행점,XH0002,일반매장,대리점,대리점,대전
55,2020-12-07,0,16,0,13,3,X50002,0.92857,X20SDXSHA3011BE270,대전은행(대-위),디스커버리 대전은행점,XH0002,일반매장,대리점,대리점,대전
56,2020-12-14,0,16,0,13,3,X50002,0.92857,X20SDXSHA3011BE270,대전은행(대-위),디스커버리 대전은행점,XH0002,일반매장,대리점,대리점,대전
57,2020-12-21,0,16,1,14,2,X50002,1.00000,X20SDXSHA3011BE270,대전은행(대-위),디스커버리 대전은행점,XH0002,일반매장,대리점,대리점,대전


## 2021 년 버키디워커 v2 

In [70]:
# BE
scskey_list = ['X21SDXSHA1111BE230','X21SDXSHA1111BE235','X21SDXSHA1111BE240','X21SDXSHA1111BE260','X21SDXSHA1111BE260','X21SDXSHA1111BE270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
be_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

In [71]:
# BK
scskey_list = ['X21SDXSHA1111BK230','X21SDXSHA1111BK235','X21SDXSHA1111BK240','X21SDXSHA1111BK260','X21SDXSHA1111BK260','X21SDXSHA1111BK270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
bk_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-60-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [72]:
# CR
scskey_list = ['X21SDXSHA1111CR230','X21SDXSHA1111CR235','X21SDXSHA1111CR240','X21SDXSHA1111CR260','X21SDXSHA1111CR260','X21SDXSHA1111CR270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
cr_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [73]:
# IV
scskey_list = ['X21SDXSHA1111IV230','X21SDXSHA1111IV235','X21SDXSHA1111IV240','X21SDXSHA1111IV260','X21SDXSHA1111IV260','X21SDXSHA1111IV270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
iv_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [74]:
# LG
scskey_list = ['X21SDXSHA1111LG230','X21SDXSHA1111LG235','X21SDXSHA1111LG240','X21SDXSHA1111LG260','X21SDXSHA1111LG260','X21SDXSHA1111LG270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
lg_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [75]:
# ow
scskey_list = ['X21SDXSHA1111OW230','X21SDXSHA1111OW235','X21SDXSHA1111OW240','X21SDXSHA1111OW260','X21SDXSHA1111OW260','X21SDXSHA1111OW270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
ow_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-60-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [76]:
# WH
scskey_list = ['X21SDXSHA1111WH230','X21SDXSHA1111WH235','X21SDXSHA1111WH240','X21SDXSHA1111WH260','X21SDXSHA1111WH260','X21SDXSHA1111WH270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
wh_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:


In [77]:
# WW
scskey_list = ['X21SDXSHA1111WW230','X21SDXSHA1111WW235','X21SDXSHA1111WW240','X21SDXSHA1111WW260','X21SDXSHA1111WW260','X21SDXSHA1111WW270']
frame = shop_weeks_buket_v2_2021_df[(shop_weeks_buket_v2_2021_df['scskey']=='X21SDXSHA1111IV240')][['shopkey','weekkey','week_takeout_qty','takeout_qty','week_sale_qty','sale_qty','stock_qty']].groupby('weekkey').sum()
v2_dwalker_shop_key_2021 = shop_weeks_buket_v2_2021_df['shopkey'].unique()


shop_weeks_buket_2021 = zero_fill(v2_dwalker_shop_key_2021, frame,scskey_list,shop_weeks_buket_v2_2021_df)
ww_shop_weeks_buket_2021_df = pd.concat(shop_weeks_buket_2021)

<ipython-input-60-b30433df00a6>:29: RuntimeWarning: divide by zero encountered in longlong_scalars
  if a['sale_qty'] / total_sale_qty > 1:
<ipython-input-60-b30433df00a6>:32: RuntimeWarning: divide by zero encountered in longlong_scalars
  frame.iloc[i] = [a[0],a[1] ,a[2] ,a[3] ,a[4] ,a[5],a[6] , a['sale_qty'] / total_sale_qty]


In [78]:
# buket
# be_shop_weeks_buket_df.to_csv('be_shop_weeks_buket_df.csv')
# wh_shop_weeks_buket_df.to_csv('wh_shop_weeks_buket_df.csv')
# bk_shop_weeks_buket_df.to_csv('bk_shop_weeks_buket_df.csv')
# gr_shop_weeks_buket_df.to_csv('gr_shop_weeks_buket_df.csv')

# v2
# ow_shop_weeks_buket_v2_df.to_csv('ow_shop_weeks_buket_v2_df.csv')
# wh_shop_weeks_buket_v2_df.to_csv('wh_shop_weeks_buket_v2_df.csv')
# be_shop_weeks_buket_v2_df.to_csv('be_shop_weeks_buket_v2_df.csv')


# 2021
# be_shop_weeks_buket_2021_df.to_csv('be_shop_weeks_buket_2021_df.csv')
# bk_shop_weeks_buket_2021_df.to_csv('bk_shop_weeks_buket_2021_df.csv')
# cr_shop_weeks_buket_2021_df.to_csv('cr_shop_weeks_buket_2021_df.csv')
# iv_shop_weeks_buket_2021_df.to_csv('iv_shop_weeks_buket_2021_df.csv')
# lg_shop_weeks_buket_2021_df.to_csv('lg_shop_weeks_buket_2021_df.csv')
# ow_shop_weeks_buket_2021_df.to_csv('ow_shop_weeks_buket_2021_df.csv')
# wh_shop_weeks_buket_2021_df.to_csv('wh_shop_weeks_buket_2021_df.csv')
# ww_shop_weeks_buket_2021_df.to_csv('ww_shop_weeks_buket_2021_df.csv')

# 컬러별 실재고에 따른 개수 비교

## 버킷디워커

In [79]:
be_shop_weeks_buket_df_zero,be_shop_qty = shop_weeks_buket_function(be_shop_weeks_buket_df)
wh_shop_weeks_buket_df_zero,wh_shop_qty = shop_weeks_buket_function(wh_shop_weeks_buket_df)
bk_shop_weeks_buket_df_zero,bk_shop_qty = shop_weeks_buket_function(bk_shop_weeks_buket_df)
gr_shop_weeks_buket_df_zero,gr_shop_qty = shop_weeks_buket_function(gr_shop_weeks_buket_df)

<ipython-input-31-a769965f5296>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-31-a769965f5296>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]
<ipython-input-31-a769965f5296>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [80]:
print(be_shop_qty[be_shop_qty['takeout_qty']>0].stock_qty.sum())
print(be_shop_qty[be_shop_qty['takeout_qty']>0].qty.sum())
print(wh_shop_qty[wh_shop_qty['takeout_qty']>0].stock_qty.sum())
print(wh_shop_qty[wh_shop_qty['takeout_qty']>0].qty.sum())
print(bk_shop_qty[bk_shop_qty['takeout_qty']>0].stock_qty.sum())
print(bk_shop_qty[bk_shop_qty['takeout_qty']>0].qty.sum())
print(gr_shop_qty[gr_shop_qty['takeout_qty']>0].stock_qty.sum())
print(gr_shop_qty[gr_shop_qty['takeout_qty']>0].qty.sum())

90490
42055.0
8443
21564.0
36439
37990.0
64873
36473.0


In [81]:
a,b= over_charge(be_shop_qty)
print(a)
print(b)

4.106959731776805
1.7886366959861268


In [82]:
a,b= over_charge(wh_shop_qty)
print(a)
print(b)

0.8778021217690871
1.7164310089117414


In [83]:
a,b= over_charge(bk_shop_qty)
print(a)
print(b)

1.7969744991546444
1.7000993982806494


In [84]:
a,b= over_charge(gr_shop_qty)
print(a)
print(b)

3.1137803600038545
1.6281771321214047


In [85]:
# 현재 재고가 0 인 지점 개수 
print(len(be_shop_weeks_buket_df_zero))
print(len(wh_shop_weeks_buket_df_zero))
print(len(bk_shop_weeks_buket_df_zero))
print(len(gr_shop_weeks_buket_df_zero))

1654
6624
4453
883


In [86]:
bk_be_zero_count = bk_stock_zero_function(be_shop_weeks_buket_df_zero)
bk_wh_zero_count = bk_stock_zero_function(wh_shop_weeks_buket_df_zero)
bk_bk_zero_count = bk_stock_zero_function(bk_shop_weeks_buket_df_zero)
bk_gr_zero_count = bk_stock_zero_function(gr_shop_weeks_buket_df_zero)

In [87]:
# 로직을 사용했을 때 실재고 반영한 재고가 0 개인 지점 개수 
print(bk_be_zero_count)
print(bk_wh_zero_count)
print(bk_bk_zero_count)
print(bk_gr_zero_count)

754
6293
3367
672


In [88]:
# 로직을 사용했을 때 실재고 반영한 재고가 0 개인 지점 개수 
(bk_be_zero_count)
(bk_wh_zero_count)
(bk_bk_zero_count)
(bk_gr_zero_count)

672

In [89]:
# print(1-(bk_be_zero_count/len(be_shop_weeks_buket_df_zero)))
print(1-(bk_be_zero_count/len(be_shop_weeks_buket_df_zero)))
print(1-(bk_wh_zero_count/len(wh_shop_weeks_buket_df_zero)))
print(1-(bk_bk_zero_count/len(bk_shop_weeks_buket_df_zero)))
print(1-(bk_gr_zero_count/len(gr_shop_weeks_buket_df_zero)))

0.5441354292623942
0.049969806763284996
0.24388052997978893
0.23895809739524354


In [90]:
((1-(bk_be_zero_count/len(be_shop_weeks_buket_df_zero)))+(1-(bk_wh_zero_count/len(wh_shop_weeks_buket_df_zero)))+(1-(bk_bk_zero_count/len(bk_shop_weeks_buket_df_zero)))+(1-(bk_gr_zero_count/len(gr_shop_weeks_buket_df_zero)))) /4

0.26923596585017795

## v2 버킷 디워커 

In [91]:
be_shop_weeks_v2_df_zero,be_v2_shop_qty = shop_weeks_buket_v2_function(ow_shop_weeks_buket_v2_df)
wh_shop_weeks_v2_df_zero,wh_v2_shop_qty =shop_weeks_buket_v2_function(wh_shop_weeks_buket_v2_df)
ow_shop_weeks_v2_df_zero,ow_v2_shop_qty = shop_weeks_buket_v2_function(be_shop_weeks_buket_v2_df)


<ipython-input-32-4bf35bc183af>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-32-4bf35bc183af>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]
<ipython-input-32-4bf35bc183af>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [92]:
print(be_v2_shop_qty[be_v2_shop_qty['takeout_qty']>0].stock_qty.sum())
print(be_v2_shop_qty[be_v2_shop_qty['takeout_qty']>0].qty.sum())
print(wh_v2_shop_qty[wh_v2_shop_qty['takeout_qty']>0].stock_qty.sum())
print(wh_v2_shop_qty[wh_v2_shop_qty['takeout_qty']>0].qty.sum())
print(ow_v2_shop_qty[ow_v2_shop_qty['takeout_qty']>0].stock_qty.sum())
print(ow_v2_shop_qty[ow_v2_shop_qty['takeout_qty']>0].qty.sum())

91095
49054.0
68173
46019.0
76890
46516.0


In [93]:
a,b= over_charge(be_v2_shop_qty)
print(a)
print(b)
a,b= over_charge(wh_v2_shop_qty)
print(a)
print(b)
a,b= over_charge(ow_v2_shop_qty)
print(a)
print(b)

3.450656814449918
1.6941520467836255
2.656609195402299
1.7241854636591478
2.9923645320197045
1.7126148705096074


In [94]:
# 현재 재고가 0 인 지점 개수 
print(len(be_shop_weeks_v2_df_zero))
print(len(wh_shop_weeks_v2_df_zero))
print(len(ow_shop_weeks_v2_df_zero))

190
227
313


In [95]:
wh_shop_weeks_v2_df_zero

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
9,2020-01-20,0,1,1,1,0,X50088,0.33333,X20SDXSHA1011WH230,진접(대-위),디스커버리 진접점,0.66668,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
9,2020-01-20,0,1,1,1,0,X50088,0.16667,X20SDXSHA1011WH235,진접(대-위),디스커버리 진접점,0.66668,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
17,2020-03-16,0,3,2,3,0,X50088,0.50000,X20SDXSHA1011WH235,진접(대-위),디스커버리 진접점,1.33343,XH0096,일반매장,대리점,대리점,경기,0.00030,2.00000
4,2019-12-16,0,1,1,1,0,X50088,1.00000,X20SDXSHA1011WH240,진접(대-위),디스커버리 진접점,0.66942,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
9,2020-01-20,0,1,1,1,0,X50088,1.00000,X20SDXSHA1011WH260,진접(대-위),디스커버리 진접점,0.66668,XH0096,일반매장,대리점,대리점,경기,0.00000,1.00000
54,2020-11-30,-1,1,0,1,0,X50088,1.00000,X20SDXSHA1011WH260,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,2.00000
55,2020-12-07,0,1,0,1,0,X50088,1.00000,X20SDXSHA1011WH260,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,2.00000
56,2020-12-14,0,1,0,1,0,X50088,1.00000,X20SDXSHA1011WH260,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,2.00000
57,2020-12-21,0,1,0,1,0,X50088,1.00000,X20SDXSHA1011WH260,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,2.00000
58,2020-12-28,0,1,0,1,0,X50088,1.00000,X20SDXSHA1011WH260,진접(대-위),디스커버리 진접점,0.00000,XH0096,일반매장,대리점,대리점,경기,0.00000,2.00000


In [96]:
v2_be_zero_count = v2_stock_zero_function(be_shop_weeks_v2_df_zero)
v2_wh_zero_count = v2_stock_zero_function(wh_shop_weeks_v2_df_zero)
v2_ow_zero_count = v2_stock_zero_function(ow_shop_weeks_v2_df_zero)

In [97]:
# 로직을 사용했을 때 실재고 반영한 재고가 0 개인 지점 개수 
print(v2_be_zero_count)
print(v2_wh_zero_count)
print(v2_ow_zero_count)


180
210
307


In [98]:
print(1-(v2_be_zero_count/len(be_shop_weeks_v2_df_zero)))
print(1-(v2_wh_zero_count/len(wh_shop_weeks_v2_df_zero)))
print(1-(v2_ow_zero_count/len(ow_shop_weeks_v2_df_zero)))


0.052631578947368474
0.07488986784140972
0.019169329073482455


In [99]:
((1-(v2_be_zero_count/len(be_shop_weeks_v2_df_zero))) + (1-(v2_wh_zero_count/len(wh_shop_weeks_v2_df_zero))) + (1-(v2_ow_zero_count/len(ow_shop_weeks_v2_df_zero))) ) / 3

0.04889692528742021

## 2021 v2 버킷디워커

In [100]:
bk_shop_weeks_buket_2021_df.head(100)
# [(be_shop_weeks_buket_2021_df['takeout_qty']>0)]

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm
0,2020-12-07,0,0,0,0,0,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
1,2020-12-14,11,11,0,0,11,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
2,2020-12-21,0,11,0,0,11,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
3,2020-12-28,0,11,0,0,11,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
4,2021-01-04,0,11,0,0,11,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
5,2021-01-11,0,11,0,0,11,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
6,2021-01-18,0,11,0,0,11,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
7,2021-01-25,-1,10,0,0,10,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
8,2021-02-01,0,10,0,0,10,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울
9,2021-02-08,0,10,0,0,10,X10052,0.00000,X21SDXSHA1111BK230,롯데강남,디스커버리 롯데 강남점,XH0254,일반매장,백화점특정,백화점,서울


In [101]:
be_shop_weeks_buket_2021_df_zero,be_2021_shop_qty = shop_weeks_buket_v2_function(be_shop_weeks_buket_2021_df)
bk_shop_weeks_buket_2021_df_zero,bk_2021_shop_qty = shop_weeks_buket_v2_function(bk_shop_weeks_buket_2021_df)
cr_shop_weeks_buket_2021_df_zero,cr_2021_shop_qty = shop_weeks_buket_v2_function(cr_shop_weeks_buket_2021_df)
iv_shop_weeks_buket_2021_df_zero,iv_2021_shop_qty = shop_weeks_buket_v2_function(iv_shop_weeks_buket_2021_df)
lg_shop_weeks_buket_2021_df_zero,lg_2021_shop_qty = shop_weeks_buket_v2_function(lg_shop_weeks_buket_2021_df)
ow_shop_weeks_buket_2021_df_zero,ow_2021_shop_qty = shop_weeks_buket_v2_function(ow_shop_weeks_buket_2021_df)
wh_shop_weeks_buket_2021_df_zero,wh_2021_shop_qty = shop_weeks_buket_v2_function(wh_shop_weeks_buket_2021_df)
ww_shop_weeks_buket_2021_df_zero,ww_2021_shop_qty = shop_weeks_buket_v2_function(ww_shop_weeks_buket_2021_df)

<ipython-input-32-4bf35bc183af>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['amount_2ewm_week'] = ewm_value[:-1]
<ipython-input-32-4bf35bc183af>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniqe_shop_scskey['qty'] = ewm_value[:-1]
<ipython-input-32-4bf35bc183af>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [102]:
print(be_2021_shop_qty[be_2021_shop_qty['takeout_qty']>0].stock_qty.sum())
print(be_2021_shop_qty[be_2021_shop_qty['takeout_qty']>0].qty.sum())
print(bk_2021_shop_qty[bk_2021_shop_qty['takeout_qty']>0].stock_qty.sum())
print(bk_2021_shop_qty[bk_2021_shop_qty['takeout_qty']>0].qty.sum())
print(cr_2021_shop_qty[cr_2021_shop_qty['takeout_qty']>0].stock_qty.sum())
print(cr_2021_shop_qty[cr_2021_shop_qty['takeout_qty']>0].qty.sum())
print(iv_2021_shop_qty[iv_2021_shop_qty['takeout_qty']>0].stock_qty.sum())
print(iv_2021_shop_qty[iv_2021_shop_qty['takeout_qty']>0].qty.sum())
print(lg_2021_shop_qty[lg_2021_shop_qty['takeout_qty']>0].stock_qty.sum())
print(lg_2021_shop_qty[lg_2021_shop_qty['takeout_qty']>0].qty.sum())
print(ow_2021_shop_qty[ow_2021_shop_qty['takeout_qty']>0].stock_qty.sum())
print(ow_2021_shop_qty[ow_2021_shop_qty['takeout_qty']>0].qty.sum())
print(wh_2021_shop_qty[wh_2021_shop_qty['takeout_qty']>0].stock_qty.sum())
print(wh_2021_shop_qty[wh_2021_shop_qty['takeout_qty']>0].qty.sum())
print(ww_2021_shop_qty[ww_2021_shop_qty['takeout_qty']>0].stock_qty.sum())
print(ww_2021_shop_qty[ww_2021_shop_qty['takeout_qty']>0].qty.sum())

299
237.0
4367
3590.0
5353
6917.0
8182
12889.0
6725
8359.0
4721
4671.0
2464
3418.0
7689
10473.0


In [103]:
def over_charge(bk_shop_qty):
    bk_shopkey_unique = bk_shop_qty.shopkey.unique()
    bk_scskey_unique = bk_shop_qty.scskey.unique()
    bk_ori_mae_list=[]
    bk_imp_mae_list=[]

    bk_score_list = []
    bk_score_list2 = []
    for i in bk_shopkey_unique:
        for j in bk_scskey_unique:
#             count = 0
#             for z in range(0,len(bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)])):
#                 if bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].takeout_qty.iloc[z]==0:
#                     count +=1
#                 else:
#                     break
# #             else:
# #                 if bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].takeout_qty.iloc[z]==0:
# #                     continue
#             else:
#                 print(count)
            bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j) & (bk_shop_qty['takeout_qty']>0)]

            bk_ori_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[count+1:] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[count:-1] ))
            bk_imp_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[count+1:-1] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[count+1:-1] ))

            bk_score_list.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[count+1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[count:-1] ).dropna()))
            bk_score_list2.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[count+1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[count+1:] ).dropna()))
    return np.mean(bk_ori_mae_list),np.mean(bk_imp_mae_list)
    

In [104]:
def over_charge(bk_shop_qty):
    bk_shopkey_unique = bk_shop_qty.shopkey.unique()
    bk_scskey_unique = bk_shop_qty.scskey.unique()
    bk_ori_mae_list=[]
    bk_imp_mae_list=[]

    bk_score_list = []
    bk_score_list2 = []
    for i in bk_shopkey_unique:
        for j in bk_scskey_unique:
#             count = 0
#             for z in range(0,len(bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)])):
#                 if bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].takeout_qty.iloc[z]==0:
#                     count +=1
#                 else:
#                     break
# #             else:
# #                 if bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].takeout_qty.iloc[z]==0:
# #                     continue
#             else:
#                 print(count)
            bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j) & (bk_shop_qty['sale_qty']>0)]

            bk_ori_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[:-1] ))
            bk_imp_mae_list.append(mae( bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:-1] ,bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[1:-1] ))

            bk_score_list.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].stock_qty[:-1] ).dropna()))
            bk_score_list2.append ( sum((bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].week_takeout_qty[1:]-bk_shop_qty[(bk_shop_qty['shopkey']==i) & (bk_shop_qty['scskey']==j)].qty[1:] ).dropna()))
    return np.mean(bk_ori_mae_list),np.mean(bk_imp_mae_list)
    

In [105]:
a,b= over_charge(be_2021_shop_qty)
print(a)
print(b)

0.04820127241179874
1.006123642439432


In [106]:
be_2021_shop_qty[(be_2021_shop_qty['shopkey']=='X10052') & (be_2021_shop_qty['scskey']=='X21SDXSHA1111BE230')].takeout_qty[:]

Series([], Name: takeout_qty, dtype: int64)

In [107]:
a,b= over_charge(be_2021_shop_qty)
print(a)
print(b)
a,b= over_charge(bk_2021_shop_qty)
print(a)
print(b)
a,b= over_charge(cr_2021_shop_qty)
print(a)
print(b)
a,b= over_charge(iv_2021_shop_qty)
print(a)
print(b)
a,b= over_charge(lg_2021_shop_qty)
print(a)
print(b)
a,b= over_charge(ow_2021_shop_qty)
print(a)
print(b)
a,b= over_charge(wh_2021_shop_qty)
print(a)
print(b)
a,b= over_charge(ww_2021_shop_qty)
print(a)
print(b)

0.04820127241179874
1.006123642439432
0.5214304993252361
1.030685045948204
0.7105995758627337
1.1570426065162909
1.1024330055909004
1.467861319966583
0.8596336996336996
1.1275856307435255
0.5179834200886831
1.1063575605680869
0.3124812030075188
1.0627234753550543
1.03837671100829
1.2688721804511278


In [108]:
len(be_shop_weeks_buket_2021_df_zero)

12

In [109]:
be_2021_shop_qty[be_2021_shop_qty['takeout_qty']>0].stock_qty.sum()

299

In [110]:
be_2021_shop_qty[be_2021_shop_qty['takeout_qty']>0].qty.sum()

237.0

In [111]:
ww_2021_shop_qty[ww_2021_shop_qty['takeout_qty']>0].stock_qty.sum()

7689

In [112]:
ww_2021_shop_qty[ww_2021_shop_qty['takeout_qty']>0].qty.sum()

10473.0

In [113]:
ow_2021_shop_qty[ow_2021_shop_qty['takeout_qty']>0].stock_qty.sum()

4721

In [114]:
ow_2021_shop_qty[ow_2021_shop_qty['takeout_qty']>0].qty.sum()

4671.0

In [115]:
be_2021_shop_qty

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
0,2020-12-07,0,0,0,0,0,X50047,0.00000,X21SDXSHA1111BE230,순천중앙(대-위),디스커버리 순천중앙점,0.00000,XH0048,일반매장,대리점,대리점,전남,0.00000,1.00000
1,2020-12-14,0,0,0,0,0,X50047,0.00000,X21SDXSHA1111BE230,순천중앙(대-위),디스커버리 순천중앙점,0.00000,XH0048,일반매장,대리점,대리점,전남,0.00000,1.00000
2,2020-12-21,0,0,0,0,0,X50047,0.00000,X21SDXSHA1111BE230,순천중앙(대-위),디스커버리 순천중앙점,0.00000,XH0048,일반매장,대리점,대리점,전남,0.00000,1.00000
3,2020-12-28,0,0,0,0,0,X50047,0.00000,X21SDXSHA1111BE230,순천중앙(대-위),디스커버리 순천중앙점,0.00000,XH0048,일반매장,대리점,대리점,전남,0.00000,1.00000
4,2021-01-04,0,0,0,0,0,X50047,0.00000,X21SDXSHA1111BE230,순천중앙(대-위),디스커버리 순천중앙점,0.00000,XH0048,일반매장,대리점,대리점,전남,0.00000,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,2021-03-22,0,0,0,0,0,X50096,0.00000,X21SDXSHA1111BE270,LF스퀘어양주(대-위),디스커버리 LF스퀘어 양주점,0.00000,XH0084,일반매장,대리점,대리점,경기,0.00000,1.00000
16,2021-03-29,0,0,0,0,0,X50096,0.00000,X21SDXSHA1111BE270,LF스퀘어양주(대-위),디스커버리 LF스퀘어 양주점,0.00000,XH0084,일반매장,대리점,대리점,경기,0.00000,1.00000
17,2021-04-05,0,0,-1,-1,1,X50096,1.00000,X21SDXSHA1111BE270,LF스퀘어양주(대-위),디스커버리 LF스퀘어 양주점,-0.66667,XH0084,일반매장,대리점,대리점,경기,0.00000,1.00000
18,2021-04-12,0,0,0,-1,1,X50096,1.00000,X21SDXSHA1111BE270,LF스퀘어양주(대-위),디스커버리 LF스퀘어 양주점,-0.22222,XH0084,일반매장,대리점,대리점,경기,-0.66667,1.00000


## 2021년 버킷디워커 v2

In [119]:
v2_2021_be_zero_count = v2_2021_stock_zero_function(be_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
v2_2021_be_zero_count

(1, [])

In [120]:
wh_shop_weeks_buket_2021_df_zero

,weekkey,week_takeout_qty,takeout_qty,week_sale_qty,sale_qty,stock_qty,shopkey,ratio,scskey,shop_nm,shop_nm_sfdc,amount_2ewm,repr_shopcode,type_shop_nm,type_contract_nm,type_zone_nm,region_nm,amount_2ewm_week,qty
15,2021-03-22,1,1,1,1,0,X50047,0.50000,X21SDXSHA1111WH230,순천중앙(대-위),디스커버리 순천중앙점,0.66667,XH0048,일반매장,대리점,대리점,전남,0.00000,1.00000
16,2021-03-29,0,1,0,1,0,X50047,0.50000,X21SDXSHA1111WH230,순천중앙(대-위),디스커버리 순천중앙점,0.22222,XH0048,일반매장,대리점,대리점,전남,0.66667,2.00000
18,2021-04-12,0,2,1,2,0,X50047,1.00000,X21SDXSHA1111WH230,순천중앙(대-위),디스커버리 순천중앙점,0.69136,XH0048,일반매장,대리점,대리점,전남,0.07407,2.00000
18,2021-04-12,0,1,1,1,0,X50047,1.00000,X21SDXSHA1111WH260,순천중앙(대-위),디스커버리 순천중앙점,0.66667,XH0048,일반매장,대리점,대리점,전남,0.00000,1.00000
18,2021-04-12,0,1,1,1,0,X50047,1.00000,X21SDXSHA1111WH260,순천중앙(대-위),디스커버리 순천중앙점,0.66667,XH0048,일반매장,대리점,대리점,전남,0.00000,2.00000
15,2021-03-22,-1,1,0,1,0,X50047,1.00000,X21SDXSHA1111WH270,순천중앙(대-위),디스커버리 순천중앙점,0.00003,XH0048,일반매장,대리점,대리점,전남,0.00010,2.00000
16,2021-03-29,0,1,0,1,0,X50047,1.00000,X21SDXSHA1111WH270,순천중앙(대-위),디스커버리 순천중앙점,0.00001,XH0048,일반매장,대리점,대리점,전남,0.00003,2.00000
5,2021-01-11,0,1,1,1,0,X50107,0.50000,X21SDXSHA1111WH270,제천(대-위),디스커버리 제천점,0.66758,XH0063,일반매장,대리점,대리점,충북,0.00000,1.00000
15,2021-03-22,-1,2,0,2,0,X50107,1.00000,X21SDXSHA1111WH270,제천(대-위),디스커버리 제천점,0.00011,XH0063,일반매장,대리점,대리점,충북,0.00034,2.00000
16,2021-03-29,0,2,0,2,0,X50107,1.00000,X21SDXSHA1111WH270,제천(대-위),디스커버리 제천점,0.00004,XH0063,일반매장,대리점,대리점,충북,0.00011,2.00000


In [121]:
print(len(be_shop_weeks_buket_2021_df_zero))
print(len(bk_shop_weeks_buket_2021_df_zero))
print(len(cr_shop_weeks_buket_2021_df_zero))
print(len(iv_shop_weeks_buket_2021_df_zero))
print(len(lg_shop_weeks_buket_2021_df_zero))
print(len(ow_shop_weeks_buket_2021_df_zero))
print(len(wh_shop_weeks_buket_2021_df_zero))
print(len(ww_shop_weeks_buket_2021_df_zero))


12
76
305
1459
292
270
445
646


In [130]:
v2_2021_be_zero_count = v2_2021_stock_zero_function(be_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
v2_2021_bk_zero_count = v2_2021_stock_zero_function(bk_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
v2_2021_cr_zero_count = v2_2021_stock_zero_function(cr_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
v2_2021_iv_zero_count = v2_2021_stock_zero_function(iv_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
v2_2021_lg_zero_count = v2_2021_stock_zero_function(lg_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
v2_2021_ow_zero_count = v2_2021_stock_zero_function(ow_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
v2_2021_wh_zero_count = v2_2021_stock_zero_function(wh_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
v2_2021_ww_zero_count = v2_2021_stock_zero_function(ww_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)


In [131]:
print(v2_2021_be_zero_count)
print(v2_2021_bk_zero_count)
print(v2_2021_cr_zero_count)
print(v2_2021_iv_zero_count)
print(v2_2021_lg_zero_count)
print(v2_2021_ow_zero_count)
print(v2_2021_wh_zero_count)
print(v2_2021_ww_zero_count)

1
63
210
1055
239
148
331
426


In [132]:
print( v2_2021_be_zero_count / len(be_shop_weeks_buket_2021_df_zero))
print(v2_2021_bk_zero_count)
print(v2_2021_cr_zero_count)
print(v2_2021_iv_zero_count)
print(v2_2021_lg_zero_count)
print(v2_2021_ow_zero_count)
print(v2_2021_wh_zero_count)
print(v2_2021_ww_zero_count)

0.08333333333333333
63
210
1055
239
148
331
426


In [133]:
print(1-(v2_2021_be_zero_count/ len(be_shop_weeks_buket_2021_df_zero)))
print(1-(v2_2021_bk_zero_count/len(bk_shop_weeks_buket_2021_df_zero)))
print(1-(v2_2021_cr_zero_count/len(cr_shop_weeks_buket_2021_df_zero)))
print(1-(v2_2021_iv_zero_count/len(iv_shop_weeks_buket_2021_df_zero)))
print(1-(v2_2021_lg_zero_count/len(lg_shop_weeks_buket_2021_df_zero)))
print(1-(v2_2021_ow_zero_count/len(ow_shop_weeks_buket_2021_df_zero)))
print(1-(v2_2021_wh_zero_count/len(wh_shop_weeks_buket_2021_df_zero)))
print(1-(v2_2021_ww_zero_count/len(ww_shop_weeks_buket_2021_df_zero)))

0.9166666666666666
0.17105263157894735
0.3114754098360656
0.27690198766278273
0.18150684931506844
0.45185185185185184
0.25617977528089886
0.3405572755417957


In [134]:
((1-(v2_2021_be_zero_count/ len(be_shop_weeks_buket_2021_df_zero))) + (1-(v2_2021_bk_zero_count/len(bk_shop_weeks_buket_2021_df_zero)))+(1-(v2_2021_cr_zero_count/len(cr_shop_weeks_buket_2021_df_zero)))+(1-(v2_2021_iv_zero_count/len(iv_shop_weeks_buket_2021_df_zero)))+(1-(v2_2021_lg_zero_count/len(lg_shop_weeks_buket_2021_df_zero)))+(1-(v2_2021_ow_zero_count/len(ow_shop_weeks_buket_2021_df_zero)))+(1-(v2_2021_wh_zero_count/len(wh_shop_weeks_buket_2021_df_zero)))+(1-(v2_2021_ww_zero_count/len(ww_shop_weeks_buket_2021_df_zero)))) / 8

0.36327405596675966

In [138]:
def count_value(ori,frame):
    ori = ori.groupby(['weekkey']).count().qty.reset_index()
    frame = frame.groupby('weekkey').count().reset_index()[['weekkey','stock_qty']]
    frame.stock_qty = 0
    ori['weekkey'] = pd.to_datetime(ori['weekkey'])

    for i in range(0,len(frame)):
        for j in range(0,len(ori)):
            if frame.iloc[i].weekkey == ori.iloc[j].weekkey:
                frame.iloc[i] = [ori.iloc[j].weekkey , ori.iloc[j].qty]
    return frame
# bk_zero_ori_frame.to_csv('bk_zero_ori_frame.csv')

In [139]:
# 2019
a,b = v2_2021_stock_zero_function(be_shop_weeks_buket_df_zero,bk_stock_scsweek_pd)
df_s = pd.DataFrame(b)
frame1 = count_value(df_s,bk_stock_scsweek_pd)
frame2 = count_value(be_shop_weeks_buket_df_zero,bk_stock_scsweek_pd)
frame2['qty']=(frame2.stock_qty- frame1.stock_qty)
# frame2[['weekkey','qty']].to_csv('2019_zero_point.csv')

TypeError: cannot unpack non-iterable int object

In [140]:
# 2020
a,b = v2_2021_stock_zero_function(be_shop_weeks_v2_df_zero,v2_stock_scsweek_pd)
df_s = pd.DataFrame(b)
frame1 = count_value(df_s,bk_stock_scsweek_pd)
frame2 = count_value(be_shop_weeks_v2_df_zero,v2_stock_scsweek_pd)
frame2['qty']=(frame2.stock_qty- frame1.stock_qty)
# frame2[['weekkey','qty']].to_csv('2020_zero_point.csv')

TypeError: cannot unpack non-iterable int object

In [141]:
# 2021
a,b = v2_2021_stock_zero_function(bk_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
df_s = pd.DataFrame(b)
frame1 = count_value(df_s,bk_stock_scsweek_pd)
frame2 = count_value(bk_shop_weeks_buket_2021_df_zero,v2_2021_stock_scsweek_pd)
frame2['qty']=(frame2.stock_qty- frame1.stock_qty)
# frame2[['weekkey','qty']].to_csv('2021_zero_point.csv')

TypeError: cannot unpack non-iterable int object